In [10]:
from __future__ import absolute_import, division, print_function


import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [11]:
# MNIST dataset parameters.

num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training Parameters

learning_rate = 0.001
training_steps = 1000
batch_size = 32
display_step = 100

# Network Parameters

num_input = 28 # number of sequences.
timesteps = 28 # timesteps.
num_units = 32 # number of neurons for the LSTM layer.

In [12]:
# Prepare MNIST data.

from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, 28, 28]), x_test.reshape([-1, num_features])

x_train, x_test = x_train / 255., x_test / 255.

In [14]:
class LSTM(Model):
    # Set layers.
    def __init__(self):
        super(LSTM, self).__init__()
        # RNN (LSTM) hidden layer.
        self.lstm_layer = layers.LSTM(units=num_units)
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        # LSTM layer.
        x = self.lstm_layer(x)
        
        x = self.out(x)          # Output layer (num_classes).
        if not is_training:
            x = tf.nn.softmax(x) # apply softmax when not training.
        return x

# Build LSTM model.
lstm_net = LSTM()

In [15]:
# Cross-Entropy Loss.
#apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Adam optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [16]:
def run_optimization(x, y):
    
    with tf.GradientTape() as g:
        # Forward pass.
        pred = lstm_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update
    trainable_variables = lstm_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update weights following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [17]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = lstm_net(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 1.833434, accuracy: 0.437500
step: 200, loss: 1.234630, accuracy: 0.593750
step: 300, loss: 1.141386, accuracy: 0.593750
step: 400, loss: 0.810610, accuracy: 0.781250
step: 500, loss: 0.365388, accuracy: 0.968750
step: 600, loss: 0.417440, accuracy: 0.875000
step: 700, loss: 0.364808, accuracy: 0.875000
step: 800, loss: 0.606892, accuracy: 0.843750
step: 900, loss: 0.481163, accuracy: 0.875000
step: 1000, loss: 0.138344, accuracy: 0.968750
